In [1]:
import networkx as nx
import numpy as np
import scipy.sparse as sp_sparse

import pandas as pd

import matplotlib.pyplot as plt
#import vispy.plot as plt
import sklearn as sk
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
import string as string
import sklearn.feature_extraction.text as sk_text
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import json
%matplotlib inline
import sklearn.utils as utils
from sklearn.model_selection import train_test_split

import datetime as dt

In [2]:
business={}
reviews=[]
compression_opts = dict(method='zip',
                            archive_name='out.csv')




In [3]:
friends=nx.Graph()

In [4]:
def prune():
    global business,reviews
    with open('yelp_academic_dataset_business.json',mode='r',encoding='utf-8',errors='ignore') as fbusiness:
        for jsonline in fbusiness:
            read = json.loads(jsonline)
            if read["city"] == "Toronto" and read["review_count"] >= 50  :
                business[read["business_id"]] = ''
                #business.append(read["business_id"])

    with open('yelp_academic_dataset_review.json',mode='r',encoding='utf-8',errors='ignore') as freviews:
        for jsonline in freviews:
            read = json.loads(jsonline)
            if read['business_id'] in business:
                dct = {
                'user_id':read['user_id'],
                'business_id':read['business_id'],
                #'date':dt.datetime.strptime(read['date'],"%Y-%m-%d %H:%M:%S"),
                'rating':read['stars']
                }
                reviews.append(dct)

    reviews=pd.DataFrame(reviews)

    reviews['freq'] = reviews.groupby('user_id')['user_id'].transform('count')
    reviews.drop(reviews.index[reviews['freq'] < 20], inplace = True)
    reviews['freq'] = reviews.groupby('business_id')['business_id'].transform('count')
    reviews.drop(reviews.index[reviews['freq'] < 50], inplace = True)
    reviews['freq'] = reviews.groupby('user_id')['user_id'].transform('count')
    reviews.drop(reviews.index[reviews['freq'] < 20], inplace = True)
    reviews['freq'] = reviews.groupby('business_id')['business_id'].transform('count')
    reviews.drop(reviews.index[reviews['freq'] < 50], inplace = True)
    reviews.drop(columns='freq',inplace=True)
    del business

In [5]:
def save():
    global reviews
    global compression_opts
    reviews.to_csv('prunedex4.zip',index=False,compression=compression_opts)

In [6]:
#prune()
#save()
reviews=pd.read_csv('prunedex4.zip',compression=compression_opts)

In [7]:
reviews=pd.pivot_table(reviews,index='user_id',columns='business_id',values='rating')
reviews=reviews.astype(pd.SparseDtype(float))

In [8]:
reviews.head(15)

business_id,-76didnxGiiMO80BjSpYsQ,-9dmhyBvepc08KPEHlEM0w,-9u2uFwhlj3Yq4mG5IweoA,-BbnAc9YEO6pjvJGEtFbVQ,-ICGmF2qUVKdvOehVNgPbg,-Ipm_8YXj9UoNNHagCvsNg,-J6FVdY9pSgAdFmmalO-pQ,-OxDX2fPQLYi6ChW2Z6xxQ,-PL-4fFYPYrOjnNd0lO4Lg,-TjzQsimpriXVYaH3HdtQA,...,zFR99jgMi-qzaJXIx8MXHA,zIfCWjZ0RAe8YVMSPZdOdA,z_twvOBwyGNbuxJhsiJofg,zb61eT_ChYctlCXcIqlW6Q,zfQ855VX3SMA_54oVSN5Cw,zg5rJfgT4jhzg1d6r2twnA,zgQHtqX0gqMw1nlBZl2VnQ,zhoNuaWrB125_LRRVPWMSQ,zrYpLdnGKA_EmOhgRCy_vg,zzUj3ej4vm_DtvRxNvWDEw
user_id,,,,,,,,,,,,,,,,,,,,,
--BumyUHiO_7YsHurb9Hkw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Qh8yKWAvIP4V4K8ZPfHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN
--YhjyV-ce1nFLYxP49C5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
--cd_gA-9Q8gM9P2cTxEsQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
-1wbglcr6x1qrUbqP1YAIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2kCxY7_aw5hOz7fJnGMbQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2n9u5-z_uKuZZN6wnh5Ow,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-3Kq9ssSot4QizbPYO6ZQQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
-3PTUP443q6hQESLKSu95w,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
def genGraph():
    global friends
    global reviews
    users=set(reviews.index)
    with open('yelp_academic_dataset_user.json',mode='r',encoding='utf-8',errors='ignore') as fusers:
            for jsonline in fusers:
                read = json.loads(jsonline)
                uid=read['user_id']
                if uid in users:
                    fr=read['friends'].split(',')
                    [friends.add_edges_from([(uid,x)]) for x in fr]

    #nx.write_edgelist(friends,"friends.adjlist")

In [10]:
genGraph()
#friends=nx.read_edgelist("friends.adjlist")

nx.draw_networkx(friends)

In [11]:
largest_cc = max(nx.connected_components(friends), key=len)
#print(largest_cc)
friends = nx.subgraph(friends,largest_cc)
#nx.draw_networkx(CC_max)

In [12]:
reviews = reviews[reviews.index.isin(list(friends))]

In [13]:
D_train, D_test= train_test_split(reviews, 
test_size=0.05,
#test_size=50,
random_state=2944)

In [14]:
D_test.head(10)

business_id,-76didnxGiiMO80BjSpYsQ,-9dmhyBvepc08KPEHlEM0w,-9u2uFwhlj3Yq4mG5IweoA,-BbnAc9YEO6pjvJGEtFbVQ,-ICGmF2qUVKdvOehVNgPbg,-Ipm_8YXj9UoNNHagCvsNg,-J6FVdY9pSgAdFmmalO-pQ,-OxDX2fPQLYi6ChW2Z6xxQ,-PL-4fFYPYrOjnNd0lO4Lg,-TjzQsimpriXVYaH3HdtQA,...,zFR99jgMi-qzaJXIx8MXHA,zIfCWjZ0RAe8YVMSPZdOdA,z_twvOBwyGNbuxJhsiJofg,zb61eT_ChYctlCXcIqlW6Q,zfQ855VX3SMA_54oVSN5Cw,zg5rJfgT4jhzg1d6r2twnA,zgQHtqX0gqMw1nlBZl2VnQ,zhoNuaWrB125_LRRVPWMSQ,zrYpLdnGKA_EmOhgRCy_vg,zzUj3ej4vm_DtvRxNvWDEw
user_id,,,,,,,,,,,,,,,,,,,,,
P7EV3yrn1PLHDBX6rF6qIg,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kKsMvWY2OnNDjiDIJ4BzLA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xecuSgvqSEQ9Fee_zk6WDw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pR7D2iviWcPvZ_Zk1tJeJg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jXedsbxoX7EoiJOOBH7DJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hYnq9-wO-RzmiTTGMCNJgA,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
1HFkcIGKY7-e9vCsc8PX5g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
TJpPDsFGNm_MfO6Z3nqfDA,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
NB4-jBvOM5Xt833RxTF5cQ,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [15]:
D_train[D_train.columns[0]].dropna()

user_id
B0Xxo-DtDejGVHQccS1iiA    2.0
oLLYmkej2TMHaTbudug5eA    3.0
BMmtD4a2uTyFxDJ2qHUbSw    3.0
06TjiB7kFn_XnlhxkiEKwQ    3.0
SkpCulULgiBfLc4lV8vx2g    3.0
                         ... 
TbhyP24zYZqZ2VJZgu1wrg    4.0
VnKj044Tt5doqaVDIJbWsw    4.0
jZrbuGRk92uWTu5kHbnDXQ    2.0
J77nvoepKNnqTI7fsPAZ9g    3.0
on4PBkuSAUi8gqlJHSyefA    4.0
Name: -76didnxGiiMO80BjSpYsQ, Length: 67, dtype: Sparse[float64, nan]

In [16]:
#D_test_dict = D_test.reset_index().melt(value_name='rating',id_vars='user_id').dropna().to_dict('records')
#D_train_dict = D_train.reset_index().melt(value_name='rating',id_vars='user_id').dropna().to_dict('records')

for business in D_train.columns:
    for user in D_train[business].dropna().index:
        friends.nodes[user][business]= D_train.loc[user].at[business]
        #print(friends.nodes[user][business])

In [17]:
print(friends.nodes['VswgrudhENYcAr8KpE4JOw'].items())

dict_items([])


for business in D_test.columns:
    for user in D_test[business].dropna().index:
        friends.nodes[user][business]['old'] = 0
        friends.nodes[user][business] = 0

Initialise the values from the train set on the graph. Set the missing values to 0. Value propagation effectively means that missing values on the nodes are calculated by averaging the values between the neighbours, some of the neighbours are absorbing nodes which means they belong to D_train and have a known rating that won't change throughout the whole process

In [27]:
results=[]

In [31]:
eps = 0.0001
for business in D_test.columns:
    absorbers = D_train[business].dropna().index
    '''
    for user in absorbers:
        friends.nodes[user]['rating']= D_train.loc[user].at[business]
        friends.nodes[user]['old_r']= D_train.loc[user].at[business]
    for user in D_test[business].dropna() user not in absorbers.any():
        friends.nodes[user]['rating'] = 0
        friends.nodes[user]['old_r'] = 0
    '''
    for user in list(friends):
        if user in absorbers:
            friends.nodes[user]['rating']= D_train.loc[user].at[business]
            friends.nodes[user]['old_r']= D_train.loc[user].at[business]
        else:
           friends.nodes[user]['rating'] = 0
           friends.nodes[user]['old_r'] = 0 
    while(True):
        for(x,y) in friends.edges:
            if x not in absorbers:
                friends.nodes[x]['rating'] += friends.nodes[y]['old_r'] / friends.degree(y)
            if y not in absorbers:
                friends.nodes[y]['rating'] += friends.nodes[x]['old_r'] / friends.degree(x)
        diff = 0
        for x in friends.nodes:
            if x not in absorbers:
                diff = max(diff, abs(friends.nodes[x]['rating'] - friends.nodes[x]['old_r']))
        if diff<=eps: break

        for x in friends:
            if x not in absorbers:
                friends.nodes[x]['old_r'] = friends.nodes[x]['rating']
                friends.nodes[x]['rating'] = 0
    
    print({x:friends.nodes[x]['rating'] for x in friends.nodes})
    results.append([{'user_id': x , 'business_id':business, 'rating':friends.nodes[x]['rating']} for x in D_test[business].dropna().index])

NameError: name 'karate' is not defined

I ran out of time and I'm not sure if the above is correct or falls into some endless loop.
~Miltiades Vasiliades 2944